# Interacting with ionic species representations using ElementEmbeddings

This notebook will serve as a tutorial for using the ElementEmbeddings package to interact with ionic species representations.

In [ ]:
from elementembeddings.core import SpeciesEmbedding
from elementembeddings.composition import (
    SpeciesCompositionalEmbedding,
    species_composition_featuriser,
)

Elements are the building blocks of chemistry, but species (elements in a given charge state) dictate the structure and properties of inorganic compounds. 

For example, the local spin and atomic environment in Fe(s), FeO, Fe2O3, and Fe3O4 solids are different due to variations in the charge state and coordination of iron.

For composition only machine learning, there many representation schemes that enable us to represent compounds as vectors, built on embeddings of elements. However, this may present a limitation when we want to represent ionic species, as the charge state of the element is not taken into account. As such, we need to represent ionic species as vectors.

The ElementEmbeddings package contains a set of pre-trained embeddings for elements and ionic species, which can be used to represent ionic species in a vector space.

At the time of writing, the 200-dimension SkipSpecies vector embeddings are available for ionic species representations. These embeddings are trained using the Skip-gram model on a large dataset of inorganic compounds.

In [ ]:
# Load the SkipSpecies vectors as a SpeciesEmbedding object

skipspecies = SpeciesEmbedding.load_data(embedding_name="skipspecies")


print("Below is the representation of Fe3+ using the SkipSpecies vectors.")

print(skipspecies.embeddings["Fe3+"])

We can check the ionic species which have a feature vector for a particular embedding

In [ ]:
print("SkipSpecies has feature vectors for the following ionic species:\n")
print(skipspecies.species_list)

We can also check which elements have an ionic species representation in the embedding

In [ ]:
print("The folliowing elements have SkipSpecies ionic species representations:\n")
print(skipspecies.element_list)

Like the element representations, BibTex citation information is available for the ionic species embeddings.

In [ ]:
print(skipspecies.citation())

## Representing ionic compositions using ElementEmbeddings

In addition to representing individual ionic species, we can also represent ionic compositions using the ElementEmbeddings package. This is useful for representing inorganic compounds as vectors. Let's take the example of Fe3O4.

Fe3O4 is a mixed-valence iron oxide, with a formula unit of Fe3O4. We pass the composition as a dicitionary in the following format:

```python
composition = {
    'Fe2+': 1,
    'Fe3+': 2,
    'O2-': 4
    }
```

In [ ]:
composition = {"Fe2+": 1, "Fe3+": 2, "O2-": 4}

Fe3O4_skipspecies = SpeciesCompositionalEmbedding(
    formula_dict=composition, embedding=skipspecies
)

A few properties are accessible from the `SpeciesCompositionalEmbedding` class

In [ ]:
# Print the pretty formula

print(Fe3O4_skipspecies.formula_pretty)

# Print the list of elements in the composition
print(Fe3O4_skipspecies.element_list)
# Print the list of ionic species in the composition
print(Fe3O4_skipspecies.species_list)


# Print the stoichiometric vector of the composition
print(Fe3O4_skipspecies.stoich_vector)

# Print the normalised stoichiometric vector of the composition
print(Fe3O4_skipspecies.norm_stoich_vector)

# Print the number of atoms
print(Fe3O4_skipspecies.num_atoms)

### Featurising compositions

We can featurise the composition using the `.feature_vector` method. This method returns the feature vector for the composition. This is identical in operation to the `CompositionEmbedding` class for featurising compositions.

The `species_composition_featuriser` can be used to featurise a list of compositions. This is useful for featurising a large number of compositions. It can also export the feature vectors to a pandas DataFrame by setting the `to_dataframe` argument to `True`.

In [ ]:
compositions = [
    {"Fe2+": 1, "Fe3+": 2, "O2-": 4},
    {"Fe3+": 2, "O2-": 3},
    {"Li+": 7, "La3+": 3, "Zr4+": 1, "O2-": 12},
    {"Cs+": 1, "Pb2+": 1, "I-": 3},
    {"Pb2+": 1, "Pb4+": 1, "O2-": 3},
]

featurised_comps_df = species_composition_featuriser(
    data=compositions, embedding="skipspecies", stats="mean", to_dataframe=True
)

featurised_comps_df

##
We can also calculate the "distance" between two compositions using their feature vectors. This can be used to determine which compositions are more similar to each other.

In [ ]:
print(
    f"The euclidean distance between Fe3O4 and Fe2O3 is {Fe3O4_skipspecies.distance({'Fe3+': 2, 'O2-': 3}, distance_metric='euclidean', stats='mean'):.2f}"
)
print(
    f"The euclidean distance between Fe3O4 and Pb2O3 is {Fe3O4_skipspecies.distance({'Pb2+': 1, 'Pb4+': 1, 'O2-': 3}, distance_metric='euclidean', stats='mean'):.2f}"
)
print(
    f"The euclidean distance between Fe3O4 and CsPbI3 is {Fe3O4_skipspecies.distance({'Cs+': 1, 'Pb2+': 1, 'I-': 3},distance_metric='euclidean', stats='mean'):.2f}"
)

Based on the mean-pooled feature vectors, we can see that Fe3O4 is closer to Fe2O3 than either Pb2O3 and CsPbI3.